Attributes of a task:

- Task name
- Number of hours in a week
- Start day and time (optional)
- End day and time (optional)
- is the task Optional
- is the duration of the task flexible (For example, if we can schedule working on a project for 10 hours instead of 12, then that's allowed)
- Min allotted duration of work
- Max allotted duration of work
- Take break before starting task
- [Skip for now] Duration of continuous work (after which we need a break)

Other constraints:
- Working window of a person
- Preferred working hours each weekday
- Preferred working hours during weekends

In [1]:
# CONSTANTS and HELPER FUNCTIONS

# This value is used for optional input parameters, in case no fixed value is supplied
default = 0

# Days in the week
num_days = 7
day_names = ["Saturday",
"Sunday",
"Monday",
"Tuesday",
"Wednesday",
"Thursday",
"Friday"]

@enum Days begin
    Saturday = 1
    Sunday = 2
    Monday = 3
    Tuesday = 4
    Wednesday = 5
    Thursday = 6
    Friday = 7
end

# Working hours in a day start at 10:30 (10:30am)
work_start = "10:30"

# Working hours end at 23:30 (11:30pm)
work_end = "23:30"

# We divide the day into 15 minute time slots
num_slots_per_day = Int((24*60)/15)

# Preferred working hours in the weekday and weekends
preferred_working_hours_weekday = 8
preferred_working_hours_weekend = 3

# We divide the day into 15 minute time slots. This function returns the
# time slot index corresponding to a given time in 24-hour HH:MM format.
# Minutes should be a multiple of 15. Time slot index of 00:00 is 1.
function convertTimeToSlotIndex(t)
    typeassert(t, String)
    hour, mins = split(t, ":")
    hour = parse(Int, hour)
    mins = parse(Int, mins)
    if mins%15 != 0
        print("Invalid time input")
    end
    return Int((hour*60 + mins)/15 + 1)
end

function convertSlotIndexToTime(idx)
    typeassert(idx, Int)
    hour = div(idx-1, 4)
    mins = ((idx-1)%4)*15
    time = ""
    if hour < 10
        time = "0"*string(hour)
    else
        time = string(hour)
    end
    time = time*":"
    if mins == 0
        time = time*"00"
    else
        time = time*string(mins)
    end
    return time
end

println(convertTimeToSlotIndex("17:30"))
println(convertSlotIndexToTime(71))

71
17:30


In [2]:
# Using DataFrames to represent tasks

using DataFrames

tasks = DataFrame(
    name = String[], 
    num_hours = Int[], 
    start_day = Int[], 
    start_time_slot = Int[],
    end_day = Int[],
    end_time_slot = Int[],
    is_optional = Bool[],
    is_flexible = Bool[],
    break_after_minutes = Int[],
    min_allotted_hours = Int[],
    max_allotted_hours = Int[],
    take_break_before_task = Bool[]
)

push!(tasks, Dict(
    :name => "Research", 
    :num_hours => 20, 
    :start_day => default, 
    :start_time_slot => default, 
    :end_day => Int(Friday), 
    :end_time_slot => convertTimeToSlotIndex("08:30"), 
    :is_optional => false, 
    :is_flexible => true, 
    :break_after_minutes => 60, 
    :min_allotted_hours => 2,
    :max_allotted_hours => 4,
    :take_break_before_task => true
))

push!(tasks, Dict(
    :name => "Edge Computing Course", 
    :num_hours => 10, 
    :start_day => default, 
    :start_time_slot => default, 
    :end_day => Int(Friday), 
    :end_time_slot => convertTimeToSlotIndex("16:00"), 
    :is_optional => false, 
    :is_flexible => true, 
    :break_after_minutes => 60, 
    :min_allotted_hours => 2,
    :max_allotted_hours => 4,
    :take_break_before_task => true
))

push!(tasks, Dict(
    :name => "Paper Reading For DB Reading Group", 
    :num_hours => 4, 
    :start_day => default, 
    :start_time_slot => default, 
    :end_day => Int(Tuesday), 
    :end_time_slot => convertTimeToSlotIndex("17:00"), 
    :is_optional => false,
    :is_flexible => false, 
    :break_after_minutes => 30, 
    :min_allotted_hours => 2,
    :max_allotted_hours => default,
    :take_break_before_task => true
))

push!(tasks, Dict(
    :name => "DB Reading Group", 
    :num_hours => 1, 
    :start_day => Int(Tuesday), 
    :start_time_slot => convertTimeToSlotIndex("17:00"), 
    :end_day => Int(Tuesday), 
    :end_time_slot => convertTimeToSlotIndex("18:00"), 
    :is_optional => false, 
    :is_flexible => false, 
    :break_after_minutes => default, 
    :min_allotted_hours => 1,
    :max_allotted_hours => default,
    :take_break_before_task => false
))

push!(tasks, Dict(
    :name => "Edge Computing - Read PyWren", 
    :num_hours => 2, 
    :start_day => default,
    :start_time_slot => default, 
    :end_day => Int(Friday), 
    :end_time_slot => convertTimeToSlotIndex("16:00"), 
    :is_optional => false, 
    :is_flexible => false, 
    :break_after_minutes => 30, 
    :min_allotted_hours => 2,
    :max_allotted_hours => default,
    :take_break_before_task => true
))

push!(tasks, Dict(
    :name => "Edge Computing - Read Google Scheduling", 
    :num_hours => 1, 
    :start_day => default, 
    :start_time_slot => default, 
    :end_day => Int(Friday), 
    :end_time_slot => convertTimeToSlotIndex("16:00"), 
    :is_optional => false, 
    :is_flexible => false, 
    :break_after_minutes => default, 
    :min_allotted_hours => 1,
    :max_allotted_hours => default,
    :take_break_before_task => true
))

,name,num_hours,start_day,start_time_slot,end_day,end_time_slot,is_optional,is_flexible,break_after_minutes,min_allotted_hours,max_allotted_hours,take_break_before_task
,String,Int64,Int64,Int64,Int64,Int64,Bool,Bool,Int64,Int64,Int64,Bool
1,Research,20,0,0,7,35,false,true,60,2,4,true
2,Edge Computing Course,10,0,0,7,65,false,true,60,2,4,true
3,Paper Reading For DB Reading Group,4,0,0,4,69,false,false,30,2,0,true
4,DB Reading Group,1,4,69,4,73,false,false,0,1,0,false
5,Edge Computing - Read PyWren,2,0,0,7,65,false,false,30,2,0,true
6,Edge Computing - Read Google Scheduling,1,0,0,7,65,false,false,0,1,0,true


In [3]:
# CHECK num_hours, min_allotted_hours, max_allotted_hours PARAMETERS OF ALL THE TASKS
num_tasks, _ = size(tasks)
num_errors = 0
for i=1:num_tasks
    task = tasks[i, :]
    min_allotted_hours = task.min_allotted_hours
    if min_allotted_hours == default
        continue
    end
    
    # max_allotted_hours should be a multiple of min_allotted_hours, or default
    max_allotted_hours = task.max_allotted_hours
    if max_allotted_hours != default && max_allotted_hours % min_allotted_hours != 0
        println(task.name, ": max_allotted_hours(=", max_allotted_hours, ") not a multiple of min_allotted_hours(=", min_allotted_hours, ")")
        num_errors += 1
    end
    
    # num_hours should be non-zero
    num_hours = task.num_hours
    if num_hours <= 0
        println(task.name, ": Invalid num_hours (supplied value=", num_hours, ")")
        num_errors += 1
        continue
    end
    
    # num_hours should be a multiple of min_allotted_hours.
    if num_hours % min_allotted_hours != 0
        println(task.name, ": num_hours(=", num_hours, ") not a multiple of min_allotted_hours(=", min_allotted_hours, ")")
        num_errors += 1
    end
end

if num_errors == 0
    println("All tasks have valid num_hours, min_allotted_hours and max_allotted_hours parameters")
end

All tasks have valid num_hours, min_allotted_hours and max_allotted_hours parameters


In [4]:
# PROBLEM MODEL

using JuMP, Gurobi

m = Model(solver=GurobiSolver(OutputFlag=0))

# We have a todo 2-d array for each task.
# todo[i][j][k] = 1 means the user should do task i on day j slot id k.
@variable(m, 0 <= todo[1:(num_tasks), 1:num_days, 1:num_slots_per_day] <= 1, Int)

# We have one more array for breaks/leisure
@variable(m, 0 <= leisure[1:num_days, 1:num_slots_per_day] <= 1, Int)

7×96 Array{Variable,2}:
 leisure[1,1]  leisure[1,2]  leisure[1,3]  …  leisure[1,95]  leisure[1,96]
 leisure[2,1]  leisure[2,2]  leisure[2,3]     leisure[2,95]  leisure[2,96]
 leisure[3,1]  leisure[3,2]  leisure[3,3]     leisure[3,95]  leisure[3,96]
 leisure[4,1]  leisure[4,2]  leisure[4,3]     leisure[4,95]  leisure[4,96]
 leisure[5,1]  leisure[5,2]  leisure[5,3]     leisure[5,95]  leisure[5,96]
 leisure[6,1]  leisure[6,2]  leisure[6,3]  …  leisure[6,95]  leisure[6,96]
 leisure[7,1]  leisure[7,2]  leisure[7,3]     leisure[7,95]  leisure[7,96]

In [5]:
# We can only be doing one type of task every slot
for j=1:num_days
    for k=1:num_slots_per_day
        @constraint(m, sum(todo[i, j, k] for i=1:num_tasks) + leisure[j, k] == 1)
    end
end

In [6]:
# Cannot work outside working hours
for j=1:num_days
    for k=1:(convertTimeToSlotIndex(work_start)-1)
        @constraint(m, leisure[j, k] == 1)
    end
end

for j=1:num_days
    for k=convertTimeToSlotIndex(work_end):num_slots_per_day
        @constraint(m, leisure[j, k] == 1)
    end
end

In [7]:
# Do not spend time on task before start day, time (if given)
for i=1:num_tasks
    task = tasks[i, :]
    start_day = task.start_day
    if start_day == default
        continue
    end

    if start_day > 1
        for j=1:start_day-1
            for k=1:num_slots_per_day
                @constraint(m, todo[i, j, k] == 0)
            end
        end
    end
    
    start_time_slot = task.start_time_slot
    j = start_day
    for k=1:start_time_slot-1
        @constraint(m, todo[i, j, k] == 0)
    end
end

In [8]:
# Do not spend time on task after end day, time (if given)
for i=1:num_tasks
    task = tasks[i, :]
    end_day = task.end_day
    if end_day == default
        continue
    end
    if end_day < 7
        for j=end_day+1:7
            for k=1:num_slots_per_day
                @constraint(m, todo[i, j, k] == 0)
            end
        end
    end
    
    end_time_slot = task.end_time_slot
    j = end_day
    for k=end_time_slot:num_slots_per_day
        @constraint(m, todo[i, j, k] == 0)
    end
end

In [9]:
# Number of time slots spent on each task during the week
@variable(m, slots_spent[1:num_tasks] >= 0, Int)

for i=1:num_tasks
    @constraint(m, slots_spent[i] == sum(todo[i, j, k] for k=1:num_slots_per_day for j=1:num_days))
end

In [10]:
# Should spend the required amount of time for tasks
for i=1:num_tasks
    task = tasks[i, :]
    if task.is_optional
        if task.is_flexible
            # Task is flexible. Can schedule for fewer hours.
            @constraint(m, slots_spent[i] <= task.num_hours*4)
        else
            # Either completely schedule the task, or do not schedule it at all
            @constraint(m, slots_spent[i]*(task.num_hours*4 - slots_spent[i]) == 0)
        end
    else
        # Task is not optional. Need to schedule for the specified hours.
        @constraint(m, slots_spent[i] == task.num_hours*4)
    end
end

In [11]:
# # Should have a leisure slot after break_after_minutes duration
# for i=1:num_tasks
#     task = tasks[i, :]
#     break_after_minutes = task.break_after_minutes
#     if break_after_minutes == default
#         continue
#     end
#     break_after_slots = Int(break_after_minutes/15)
#     for j=1:num_days
#         for k=1:num_slots_per_day-break_after_slots
#             @constraint(m, sum(todo[i, j, l] for l=k:k+break_after_slots) <= break_after_slots)
#             @constraint(m, sum(todo[i, j, l] for l=k:k+break_after_slots) +
#                 sum(leisure[j, l] for l=k:k+break_after_slots) <= break_after_slots + 1)
#         end
#     end
# end

In [12]:
# Should have a leisure slot before a task (if required)
for t1=1:num_tasks
    task1 = tasks[t1, :]
    if !task1.take_break_before_task
        continue
    end
    for t2=1:num_tasks
        if t2 == t1
            continue
        end
        for j=1:num_days
            for k=3:num_slots_per_day
                @constraint(m, todo[t1, j, k] + todo[t2, j, k-2] - leisure[j, k-1] <= 1)
            end
        end
    end
end

In [13]:
# Hours of work exceeded on each day of the week
@variable(m, slots_exceeded[1:num_days] >= 0, Int)
for j=1:num_days
    if j==Int(Saturday) || j==Int(Sunday)
        preferred_work_end = convertTimeToSlotIndex(work_start) + preferred_working_hours_weekend*4
    else
        preferred_work_end = convertTimeToSlotIndex(work_start) + preferred_working_hours_weekday*4
    end
    @constraint(m, slots_exceeded[j] == sum(todo[i, j, k] for k=preferred_work_end:num_slots_per_day for i=1:num_tasks))
end

In [14]:
# Min hours allocated while scheduling a task

# We first compute the start time of each task
@variable(m, 0 <= task_started[1:num_tasks, 1:num_days, 1:num_slots_per_day] <= 1, Int)
for i=1:num_tasks
    for j=1:num_days
        for k=2:num_slots_per_day
            @constraint(m, task_started[i, j, k] == todo[i, j, k]*(1 - todo[i, j, k-1]))
        end
        # For slot 1, the constraint is slightly different as there is no slot before this
        @constraint(m, task_started[i, j, 1] == todo[i, j, 1])
    end
end
                    
for i=1:num_tasks
    task = tasks[i,:]
    min_hours = task.min_allotted_hours
    if min_hours == default
        continue
    end
    min_slots = min_hours * 4 # there are four slots per hour
    for j=1:num_days
        for k=1:num_slots_per_day-min_slots+1
            @constraint(m, task_started[i,j,k]*(sum(todo[i,j,l] for l=k:k+min_slots-1)-min_slots) >=0 )
        end
        
        for k=num_slots_per_day-min_slots+2:num_slots_per_day
            @constraint(m, task_started[i,j,k]*(sum(todo[i,j,l] for l=k:num_slots_per_day)-min_slots) >=0 )
        end
    end
end

In [15]:
# Max hours allocated while scheduling a task

for i=1:num_tasks
    task = tasks[i,:]
    max_hours = task.max_allotted_hours
    if max_hours == default
        continue
    end
    max_slots = max_hours * 4 # there are four slots per hour
    for j=1:num_days
        for k=1:num_slots_per_day-max_slots
            @constraint(m, task_started[i,j,k]*(max_slots-sum(todo[i,j,l] for l=k:k+max_slots)) >=0 )
        end
    end
end

In [16]:
@objective(m, Min, sum(slots_exceeded[j] for j=1:num_days) +   # Number of slots exceeded over preferred hours
    sum(tasks[i, :].num_hours - slots_spent[i]/4 for i=1:num_tasks))  # Number of unfulfilled tasks
status = solve(m)

Academic license - for non-commercial use only


:Optimal

In [17]:
todo = getvalue(todo)
for i=1:num_tasks
    task = tasks[i, :]
    println(task.name, " schedule:")
    for j=1:num_days
        println(day_names[j], ": ", sum(todo[i, j, k] for k=1:num_slots_per_day)/4, " hours")
    end
    println()
end

Research schedule:
Saturday: 3.0 hours
Sunday: 2.75 hours
Monday: 0.0 hours
Tuesday: 5.5 hours
Wednesday: 5.0 hours
Thursday: 3.75 hours
Friday: 0.0 hours

Edge Computing Course schedule:
Saturday: 0.0 hours
Sunday: 0.0 hours
Monday: 0.0 hours
Tuesday: 0.0 hours
Wednesday: 2.25 hours
Thursday: 3.75 hours
Friday: 4.0 hours

Paper Reading For DB Reading Group schedule:
Saturday: 0.0 hours
Sunday: 0.0 hours
Monday: 4.0 hours
Tuesday: 0.0 hours
Wednesday: 0.0 hours
Thursday: 0.0 hours
Friday: 0.0 hours

DB Reading Group schedule:
Saturday: 0.0 hours
Sunday: 0.0 hours
Monday: 0.0 hours
Tuesday: 1.0 hours
Wednesday: 0.0 hours
Thursday: 0.0 hours
Friday: 0.0 hours

Edge Computing - Read PyWren schedule:
Saturday: 0.0 hours
Sunday: 0.0 hours
Monday: 2.0 hours
Tuesday: 0.0 hours
Wednesday: 0.0 hours
Thursday: 0.0 hours
Friday: 0.0 hours

Edge Computing - Read Google Scheduling schedule:
Saturday: 0.0 hours
Sunday: 0.0 hours
Monday: 0.0 hours
Tuesday: 0.0 hours
Wednesday: 0.0 hours
Thursday: 0.0

In [18]:
# Research Schedule
i = 1
println("Research Schedule:")
println()
for j=1:num_days
    println(day_names[j], ":")
    start_slot = 0
    end_slot = 0
    for k=convertTimeToSlotIndex(work_start):convertTimeToSlotIndex(work_end)-1
        if todo[i, j, k] == 1
            if start_slot == 0
                start_slot = end_slot = k
            else
                end_slot += 1
            end
        else
            if start_slot != 0
                println("Start time: ", convertSlotIndexToTime(start_slot), " End time: ", convertSlotIndexToTime(end_slot))
            end
            start_slot = end_slot = 0
        end
    end
    
    if start_slot != 0
        println("Start time: ", convertSlotIndexToTime(start_slot), " End time: ", convertSlotIndexToTime(end_slot))
    end
    println()
end

Research Schedule:

Saturday:
Start time: 10:30 End time: 13:15

Sunday:
Start time: 10:45 End time: 13:15

Monday:

Tuesday:
Start time: 11:15 End time: 14:00
Start time: 14:30 End time: 16:45

Wednesday:
Start time: 13:00 End time: 15:30
Start time: 16:15 End time: 18:15

Thursday:
Start time: 14:45 End time: 18:15

Friday:



In [19]:
# Number of hours exceeded
slots_exceeded = getvalue(slots_exceeded)
println("Number of hours exceeded:")
for j=1:num_days
    println(day_names[j], ": ", slots_exceeded[j])
end

Number of hours exceeded:
Saturday: 0.0
Sunday: 0.0
Monday: 0.0
Tuesday: 0.0
Wednesday: 0.0
Thursday: 0.0
Friday: 0.0
